# オイラーのarctan公式を用いた円周率の計算

オイラーは $\arctan$ 公式

$$\frac{\pi}{4} = 5\arctan \frac{1}{7} +2 \arctan \frac{3}{79}$$

を用いて1時間で円周率を20桁目まで求めたといわれています。計算時に、オイラー変換と呼ばれる以下の変換

$$\begin{align*}
\arctan x &= \frac{x}{1 +x^2}\left(1 + \frac{2}{3}\left(\frac{x^2}{1+x^2}\right) + \frac{2\cdot 4}{3 \cdot 5}\left(\frac{x^2}{1+x^2}\right)^2 + \cdots \right) \\
&= \frac{x}{1 +x^2} \sum_{n=0}^{\infty}\frac{(2n)!!}{(2n+1)!!}\left(\frac{x^2}{1+x^2}\right)^n 
\end{align*}$$

を用いたとされています。一般にこのような変換は、収束を加速させる目的で用いられることが多いようですが、この計算では

$$\frac{\frac{1}{7^2}}{1+ \frac{1}{7^2}} = \frac{2}{10^2}, \ \frac{\frac{3^2}{79^2}}{1+ \frac{3^2}{79^2}} = \frac{144}{10^5}$$

のように計算しやすい形に変換されることのほうが重要だったようです。


$m$ 項まで計算したときの誤差を計算してみましょう。$m$ までの和を $S_m(x)$ とおくと, $\frac{(2n)!!}{(2n+1)!!}$ が $n$ に関して単調減少であることから

$$\begin{align*}
|\arctan x -S_m(x)| &= \frac{x}{1 +x^2} \sum_{n=m+1}^{\infty} \frac{(2n)!!}{(2n+1)!!}\left(\frac{x^2}{1+x^2}\right)^n \\
& \leq \frac{(2m+2)!!}{(2m+3)!!} \frac{x}{1 +x^2} \sum_{n=m+1}^{\infty} \left(\frac{x^2}{1+x^2}\right)^n \\
& = \frac{(2m+2)!!}{(2m+3)!!} \frac{x}{1 +x^2} \left(\frac{x^2}{1+x^2}\right)^{m+1} \sum_{n=0}^{\infty} \left(\frac{x^2}{1+x^2}\right)^n \\
&= \frac{(2m+2)!!}{(2m+3)!!} \frac{x}{1 +x^2} \left(\frac{x^2}{1+x^2}\right)^{m+1} \left(\frac{1}{1 -\frac{x^2}{1+x^2}}\right) \\
&= \frac{(2m+2)!!}{(2m+3)!!} \frac{x}{1 +x^2} \left(\frac{x^2}{1+x^2}\right)^{m+1} (1+x^2)\\
&= \frac{(2m+2)!!}{(2m+3)!!} x \left( \frac{x^2}{1+x^2}\right)^{m+1}
\end{align*}$$

となります。 $x = \frac{1}{7}$ のとき

$$20|\arctan x -S_m(x)| \leq 20 \cdot \frac{2}{3} \cdot \frac{1}{7}\left(\frac{2}{10^2}\right)^{m+1} \leq  2 \left(\frac{2}{10^2}\right)^{m+1} $$

$x = \frac{3}{79}$ かつ $m \geq 1$ のとき

$$8|\arctan x -S_m(x)| \leq 8 \cdot \frac{2 \cdot 4}{3 \cdot 5} \frac{3}{79} \left(\frac{144}{10^5}\right)^{m+1} \leq \frac{1}{5} \left(\frac{144}{10^5}\right)^{m+1}$$

が成り立ちます。

In [5]:
from math import log10
from dataclasses import dataclass
import mpmath

@dataclass
class ArctanParam:
    coeff: int
    numer: int  # 分子
    denom: int  # 分母
    
    @property
    def x(self) -> float:
        return mpmath.mpf(self.numer) / self.denom
        
    def __post_init__(self):
        if self.coeff <= 0:
            raise ValueError(f"coeff must be greater than 0. given {self.coeff}")
        if self.numer <= 0:
            raise ValueError(f"numer must be greater than 0. given {self.numer}")
        if self.denom <= 0:
            raise ValueError(f"denom must be greater than 0. given {self.denom}")
            
arctan_7 = ArctanParam(20, 1, 7)
arctan_79 = ArctanParam(8, 3, 79)

def euler_log_error(p: ArctanParam, m: int) -> float:
    def coeff_log(m: int) -> float:
        val = 0
        for i in range(0, m + 1):
            val = val + log10((2 * m + 2) /(2 * m + 3) )
        return val
    return log10(p.coeff) + log10(p.x) + (m+1) * log10(p.x**2 / (1 + p.x**2)) + coeff_log(m)

In [4]:
for i in range(1, 100):
    elog_err = euler_log_error(arctan_7, i)
    
    if elog_err < -20:
        print(f"i = {i}, elog_err: {elog_err}")
        break
        
for i in range(1, 100):
    elog_err = euler_log_error(arctan_79, i)
    
    if elog_err < -20:
        print(f"i = {i}, elog_err: {elog_err}")
        break

i = 11, elog_err: -20.14445329990768
i = 6, elog_err: -20.61862096855419


これで20桁を求めるには $\arctan \frac{1}{7}$ の項をおおよそ12項、$\arctan \frac{3}{79}$ の項をおおよそ7項計算すればよいことがわかります。

In [14]:
from typing import Generator, Tuple

def euler_transform_series(p: ArctanParam) -> Generator[Tuple[float, int], None, None]:
    def next_term(cur_term: float, x: float, n: int) -> float:
        # a_{n+1} を返す
        return (mpmath.mpf(2 * n + 2) / (2 * n + 3)) * cur_term * (x ** 2 / (1 + x **2))
    
    def init_term(p: ArctanParam):
        x = p.x
        return mpmath.mpf(p.coeff) * (x / (1 + x **2))
    
    n = 0
    
    x = p.x
    cur_term = init_term(p)
    _sum = cur_term
    
    while True:
        yield (_sum, n)
        cur_term = next_term(cur_term, x, n)
        _sum = _sum + cur_term
        n = n + 1

In [22]:
import sys
sys.path.append('/home/jovyan/work/')
from lib.utils import print_pi, PI_50

DPS = 25
mpmath.mp.dps = DPS

def error(m: int, p: ArctanParam):
    return 10 ** euler_log_error(p, m)

param_7 = ArctanParam(20, 1, 7)
param_79 = ArctanParam(8, 3, 79)

et_7 = euler_transform_series(param_7)
et_79 = euler_transform_series(param_79)

for i in range(0, 11):
    next(et_7)
for i in range(0, 6):
    next(et_79)
    
(a7, m1) = next(et_7)
(a79, m2) = next(et_79)
    
def print_result(a7, a79, m1, m2):
    pi = a7 + a79

    e_1 = error(m1, param_7)
    e_2 = error(m2, param_79)

    E = e_1 + e_2

    print(f"m1 = {m1}, m2 = {m2}")

    print("・誤差無視")
    print_pi(pi, PI_50, DPS, _format="{pi} ({mdigit} 桁まで一致)")
    print("")

    print("・誤差考慮")
    print_pi(pi, PI_50, DPS, _format="{pi} ({mdigit} まで一致)")
    print(" ≦ π ≦ ")
    print_pi(pi + E, PI_50, DPS, _format="{pi} ({mdigit} まで一致)")
    print("")
    
    print("・誤差")
    print(E)
    
print_result(a7, a79, m1, m2)

(a79_next, m2_next) = next(et_79)
(a7_next, m1_next) = next(et_7)

print("\n========\n")
print_result(a7, a79_next, m1, m2_next)

print("\n========\n")
print_result(a7_next, a79, m1_next, m2)

print("\n========\n")
print_result(a7_next, a79_next, m1_next, m2_next)

m1 = 11, m2 = 6
・誤差無視
3.141592653589793238458500 (19 桁まで一致)

・誤差考慮
3.141592653589793238458500 (19 まで一致)
 ≦ π ≦ 
3.141592653589793238468077 (20 まで一致)

・誤差
9.576916888855133e-21


m1 = 11, m2 = 7
・誤差無視
3.141592653589793238459740 (19 桁まで一致)

・誤差考慮
3.141592653589793238459740 (19 まで一致)
 ≦ π ≦ 
3.141592653589793238466914 (20 まで一致)

・誤差
7.173912979740626e-21


m1 = 12, m2 = 6
・誤差無視
3.141592653589793238461346 (20 桁まで一致)

・誤差考慮
3.141592653589793238461346 (20 まで一致)
 ≦ π ≦ 
3.141592653589793238463896 (20 まで一致)

・誤差
2.5497622272629615e-21


m1 = 12, m2 = 7
・誤差無視
3.141592653589793238462586 (21 桁まで一致)

・誤差考慮
3.141592653589793238462586 (21 まで一致)
 ≦ π ≦ 
3.141592653589793238462733 (21 まで一致)

・誤差
1.4675831814845403e-22


ちなみに係数の $\frac{(2n)!!}{(2n+1)!!}$ は $n^{-\frac{1}{2}}$ のオーダーで $0$ に収束し、$\sqrt{n}\frac{(2n)!!}{(2n+1)!!}$ は 

$$\frac{\sqrt{\pi}}{2} = 0.886226925452757...$$

に収束するようです。

In [21]:
from  mpmath import mpf, mp
import math

DPS = 20
mp.dps = DPS

def coeff(n, prev):
    return prev * (2 * n) / (2 * n + 1)

val = mpf(1)

for i in range(1, 10**6):
    val = coeff(i, val)
    if i % (10 ** 4) == 0:
        print(f"i = {i}, val = {val}, √i * val = {math.sqrt(i) * val}")

i = 10000, val = 0.0088619369367387463062, √i * val = 0.88619369367387463062
i = 20000, val = 0.0062664531914368965769, √i * val = 0.8862103091306224393
i = 30000, val = 0.005116569582923556898, √i * val = 0.88621584780851001602
i = 40000, val = 0.0044310930859175623592, √i * val = 0.88621861718351247184
i = 50000, val = 0.0039632975729609106619, √i * val = 0.8862202788200528562
i = 60000, val = 0.003617983660448421036, √i * val = 0.88622138658255443824
i = 70000, val = 0.0033496049841783625192, √i * val = 0.88622217784383906349
i = 80000, val = 0.0031332706561093236145, √i * val = 0.88622277129109030847
i = 90000, val = 0.0029540774428731259533, √i * val = 0.88622323286193778598
i = 100000, val = 0.0028024850988951695585, √i * val = 0.88622360211909667292
i = 110000, val = 0.0026720656096402839675, √i * val = 0.88622390423890809689
i = 120000, val = 0.0025583087751610146296, √i * val = 0.88622415600563611916
i = 130000, val = 0.0024579441570280557344, √i * val = 0.88622436903917661406